# Chapter 3

## Initial Setup

In [1]:
%%time

import torch

import torch.nn as nn

CPU times: user 3.26 s, sys: 402 ms, total: 3.66 s
Wall time: 3.62 s


## Attending to Different Parts of the Input with Self-Attention

### A Simple Self-Attention Mechanism Without Trainable Weights

In [2]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89],  # Your     (x^1)
   [0.55, 0.87, 0.66],  # journey  (x^2)
   [0.57, 0.85, 0.64],  # starts   (x^3)
   [0.22, 0.58, 0.33],  # with     (x^4)
   [0.77, 0.25, 0.10],  # one      (x^5)
   [0.05, 0.80, 0.55],  # step     (x^6)
   [-0.55 * 1, -0.87 * 1, -0.66 * 1]] # - 1 * x2
)

In [3]:
# The second input token is the query.
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([ 0.9544,  1.4950,  1.4754,  0.8434,  0.7070,  1.0865, -1.4950])


In [4]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([ 0.1884,  0.2951,  0.2912,  0.1665,  0.1395,  0.2144, -0.2951])
Sum: tensor(1.0000)


In [5]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1369, 0.2351, 0.2305, 0.1225, 0.1069, 0.1562, 0.0118])
Sum: tensor(1.0000)


In [6]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1369, 0.2351, 0.2305, 0.1225, 0.1069, 0.1562, 0.0118])
Sum: tensor(1.0000)


In [7]:
# The second input token is the query.
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4301, 0.6335, 0.5538])


### Computing Attention Weights for All Input Tokens

In [8]:
attn_scores = torch.empty(6 + 1, 6 + 1)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[ 0.9995,  0.9544,  0.9422,  0.4753,  0.4576,  0.6310, -0.9544],
        [ 0.9544,  1.4950,  1.4754,  0.8434,  0.7070,  1.0865, -1.4950],
        [ 0.9422,  1.4754,  1.4570,  0.8296,  0.7154,  1.0605, -1.4754],
        [ 0.4753,  0.8434,  0.8296,  0.4937,  0.3474,  0.6565, -0.8434],
        [ 0.4576,  0.7070,  0.7154,  0.3474,  0.6654,  0.2935, -0.7070],
        [ 0.6310,  1.0865,  1.0605,  0.6565,  0.2935,  0.9450, -1.0865],
        [-0.9544, -1.4950, -1.4754, -0.8434, -0.7070, -1.0865,  1.4950]])


In [9]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[ 0.9995,  0.9544,  0.9422,  0.4753,  0.4576,  0.6310, -0.9544],
        [ 0.9544,  1.4950,  1.4754,  0.8434,  0.7070,  1.0865, -1.4950],
        [ 0.9422,  1.4754,  1.4570,  0.8296,  0.7154,  1.0605, -1.4754],
        [ 0.4753,  0.8434,  0.8296,  0.4937,  0.3474,  0.6565, -0.8434],
        [ 0.4576,  0.7070,  0.7154,  0.3474,  0.6654,  0.2935, -0.7070],
        [ 0.6310,  1.0865,  1.0605,  0.6565,  0.2935,  0.9450, -1.0865],
        [-0.9544, -1.4950, -1.4754, -0.8434, -0.7070, -1.0865,  1.4950]])


In [10]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2038, 0.1948, 0.1924, 0.1206, 0.1185, 0.1410, 0.0289],
        [0.1369, 0.2351, 0.2305, 0.1225, 0.1069, 0.1562, 0.0118],
        [0.1373, 0.2340, 0.2298, 0.1227, 0.1094, 0.1545, 0.0122],
        [0.1382, 0.1997, 0.1970, 0.1408, 0.1216, 0.1657, 0.0370],
        [0.1457, 0.1869, 0.1885, 0.1305, 0.1793, 0.1236, 0.0455],
        [0.1351, 0.2131, 0.2076, 0.1386, 0.0964, 0.1850, 0.0243],
        [0.0587, 0.0342, 0.0349, 0.0656, 0.0752, 0.0514, 0.6800]])


In [11]:
attn_weights.sum(dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [12]:
all_context_vecs = attn_weights @ inputs

print(f"Shapes: {attn_weights.shape} x {inputs.shape}")
print()
print(all_context_vecs)

Shapes: torch.Size([7, 7]) x torch.Size([7, 3])

tensor([[ 0.4134,  0.5508,  0.5432],
        [ 0.4301,  0.6335,  0.5538],
        [ 0.4310,  0.6310,  0.5521],
        [ 0.3941,  0.5744,  0.5063],
        [ 0.4209,  0.5246,  0.4727],
        [ 0.3943,  0.6134,  0.5348],
        [-0.2352, -0.4254, -0.2942]])


## Implementing Self-Attention with Trainable Weights

### Computing the Attention Weights Step by Step

In [13]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [14]:
torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [15]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [16]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([7, 2])
values.shape: torch.Size([7, 2])


In [17]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)

print(attn_score_22)

tensor(1.8524)


In [18]:
attn_scores_2 = query_2 @ keys.T

print(attn_scores_2)

tensor([ 1.2705,  1.8524,  1.8111,  1.0795,  0.5577,  1.5440, -1.8524])


In [19]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)

print(attn_weights_2)

tensor([0.1476, 0.2227, 0.2163, 0.1289, 0.0892, 0.1791, 0.0162])


In [20]:
context_vec_2 = attn_weights_2 @ values

print(context_vec_2)

tensor([0.2947, 0.7914])


### Implementing a Compact Self-Attention Python Class

In [21]:
class SelfAttention_v1(nn.Module):
    """
    Listing 3.1 A compact self-attention class.
    """
    
    def __init__(self, d_in, d_out):
        
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = attn_weights @ values

        return context_vec

In [22]:
torch.manual_seed(123)

sa_v1 = SelfAttention_v1(d_in, d_out)

print(sa_v1(inputs))

tensor([[ 0.2787,  0.7509],
        [ 0.2947,  0.7914],
        [ 0.2941,  0.7898],
        [ 0.2636,  0.7126],
        [ 0.2562,  0.6940],
        [ 0.2772,  0.7471],
        [-0.1392, -0.3375]], grad_fn=<MmBackward0>)


In [23]:
class SelfAttention_v2(nn.Module):
    """
    Listing 3.2 A self-attention class using PyTorch’s Linear layers.
    """
    
    def __init__(self, d_in, d_out, qkv_bias=False):
        
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1) 
        context_vec = attn_weights @ values
    
        return context_vec

In [24]:
torch.manual_seed(123)

sa_v2 = SelfAttention_v2(d_in, d_out)

print(sa_v2(inputs))

tensor([[-0.4124, -0.0758],
        [-0.3977, -0.0751],
        [-0.3977, -0.0751],
        [-0.3710, -0.0688],
        [-0.3857, -0.0713],
        [-0.3729, -0.0697],
        [-0.2928, -0.0484]], grad_fn=<MmBackward0>)


#### Exercise 3.1: Comparing Self-Attention v1 and V2

In [25]:
# Initialize the self-attention classes.
sav1 = SelfAttention_v1(d_in, d_out)
sav2 = SelfAttention_v2(d_in, d_out)

# Copy the weights from the second to the first class.
sav1.W_key.data = sav2.W_key.weight.data.T
sav1.W_query.data = sav2.W_query.weight.data.T
sav1.W_value.data = sav2.W_value.weight.data.T

# Compare if the two classes produce the same output.
print(sav1(inputs))
print()
print(sav2(inputs))
print()
print(sav1(inputs) - sav2(inputs))  # Should be zero.

tensor([[0.3035, 0.1430],
        [0.2954, 0.1392],
        [0.2955, 0.1392],
        [0.3274, 0.1542],
        [0.3163, 0.1491],
        [0.3236, 0.1524],
        [0.4059, 0.1912]], grad_fn=<MmBackward0>)

tensor([[0.3035, 0.1430],
        [0.2954, 0.1392],
        [0.2955, 0.1392],
        [0.3274, 0.1542],
        [0.3163, 0.1491],
        [0.3236, 0.1524],
        [0.4059, 0.1912]], grad_fn=<MmBackward0>)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<SubBackward0>)


## Hiding Future Words with Causal Attention

### Applying a Causal Attention Mask

In [26]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)

print(attn_weights)

tensor([[0.1550, 0.1591, 0.1590, 0.1404, 0.1468, 0.1425, 0.0972],
        [0.1459, 0.1560, 0.1558, 0.1438, 0.1432, 0.1473, 0.1080],
        [0.1460, 0.1560, 0.1557, 0.1437, 0.1432, 0.1473, 0.1080],
        [0.1428, 0.1486, 0.1485, 0.1441, 0.1424, 0.1461, 0.1276],
        [0.1473, 0.1521, 0.1520, 0.1429, 0.1442, 0.1448, 0.1167],
        [0.1420, 0.1493, 0.1491, 0.1445, 0.1420, 0.1470, 0.1262],
        [0.1380, 0.1290, 0.1293, 0.1400, 0.1406, 0.1367, 0.1864]],
       grad_fn=<SoftmaxBackward0>)


In [27]:
attn_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

In [28]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))

print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.]])


In [29]:
masked_simple = attn_weights * mask_simple

print(masked_simple)

tensor([[0.1550, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1459, 0.1560, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1460, 0.1560, 0.1557, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1428, 0.1486, 0.1485, 0.1441, 0.0000, 0.0000, 0.0000],
        [0.1473, 0.1521, 0.1520, 0.1429, 0.1442, 0.0000, 0.0000],
        [0.1420, 0.1493, 0.1491, 0.1445, 0.1420, 0.1470, 0.0000],
        [0.1380, 0.1290, 0.1293, 0.1400, 0.1406, 0.1367, 0.1864]],
       grad_fn=<MulBackward0>)


In [30]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
print(row_sums.shape)

masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

torch.Size([7, 1])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682, 0.0000],
        [0.1380, 0.1290, 0.1293, 0.1400, 0.1406, 0.1367, 0.1864]],
       grad_fn=<DivBackward0>)


In [31]:
masked_simple_norm.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

In [32]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
mask

tensor([[0., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.]])

In [33]:
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
masked

tensor([[ 0.3111,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.1655,  0.2602,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.1667,  0.2602,  0.2577,    -inf,    -inf,    -inf,    -inf],
        [ 0.0510,  0.1080,  0.1064,  0.0643,    -inf,    -inf,    -inf],
        [ 0.1415,  0.1875,  0.1863,  0.0987,  0.1121,    -inf,    -inf],
        [ 0.0476,  0.1192,  0.1171,  0.0731,  0.0477,  0.0966,    -inf],
        [-0.1655, -0.2602, -0.2576, -0.1445, -0.1384, -0.1790,  0.2602]],
       grad_fn=<MaskedFillBackward0>)

In [34]:
attn_weights = torch.softmax(masked / keys.shape[-1] ** 0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682, 0.0000],
        [0.1380, 0.1290, 0.1293, 0.1400, 0.1406, 0.1367, 0.1864]],
       grad_fn=<SoftmaxBackward0>)


In [35]:
attn_weights.sum(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)

### Masking Additional Attention Weights with Dropout

In [36]:
torch.manual_seed(123)

dropout = torch.nn.Dropout(1/2)
example = torch.ones(7, 7)

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2., 0.],
        [2., 0., 0., 0., 0., 0., 0.],
        [2., 0., 2., 0., 2., 2., 0.],
        [0., 0., 2., 2., 0., 0., 0.],
        [0., 2., 0., 2., 0., 0., 0.],
        [0., 0., 0., 2., 2., 2., 0.],
        [0., 2., 0., 0., 2., 0., 0.]])


In [37]:
torch.manual_seed(123)

print(attn_weights)
print()
print(dropout(attn_weights))

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4833, 0.5167, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3190, 0.3408, 0.3402, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2445, 0.2545, 0.2542, 0.2468, 0.0000, 0.0000, 0.0000],
        [0.1994, 0.2060, 0.2058, 0.1935, 0.1953, 0.0000, 0.0000],
        [0.1624, 0.1709, 0.1706, 0.1654, 0.1625, 0.1682, 0.0000],
        [0.1380, 0.1290, 0.1293, 0.1400, 0.1406, 0.1367, 0.1864]],
       grad_fn=<SoftmaxBackward0>)

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9665, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6380, 0.0000, 0.6804, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5085, 0.4936, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4120, 0.0000, 0.3869, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.3308, 0.3249, 0.3363, 0.0000],
        [0.0000, 0.2581, 0.0000, 0.0000, 0.2813, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


### Implementing a Compact Causal Attention Class

In [38]:
batch = torch.stack((inputs, inputs), dim=0)

print(batch.shape)

torch.Size([2, 7, 3])


In [39]:
class CausalAttention(nn.Module):
    """
    Listing 3.3: A compact causal attention class.
    """

    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):

        super().__init__()
        self.d_out = d_out
        
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)

        self.register_buffer(
            name="mask",
            tensor=torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
       
        # In PyTorch, operations with a trailing underscore are performed 
        # inplace, avoiding unnecessary memory copies.
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
        )
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)   
        attn_weights = self.dropout(attn_weights)
        context_vec = attn_weights @ values

        return context_vec

In [40]:
torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)

print("batch.shape:", batch.shape)
print("context_vecs.shape:", context_vecs.shape)

batch.shape: torch.Size([2, 7, 3])
context_vecs.shape: torch.Size([2, 7, 2])


## Extending Single-Head Attention to Multi-Head Attention

### Stacking Multiple Single-Head Attention Layers

In [41]:
class MultiHeadAttentionWrapper(nn.Module):
    """
    Listing 3.4: A wrapper class to implement multi-head attention.
    """

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)]
        )

    def forward(self, x):
        
        return torch.cat(
            [head(x) for head in self.heads], dim=-1
        )

In [42]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens.

d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493],
         [-0.2928, -0.0484,  0.3210,  0.2033]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493],
         [-0.2928, -0.0484,  0.3210,  0.2033]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 7, 4])


#### Exercise 3.2: Returning Two-Dimensional Embedding Vectors

In [43]:
torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens.

d_in, d_out = 3, 1
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065],
         [-0.4246, -0.0681]],

        [[-0.5740,  0.2216],
         [-0.7320,  0.0155],
         [-0.7774, -0.0546],
         [-0.6979, -0.0817],
         [-0.6538, -0.0957],
         [-0.6424, -0.1065],
         [-0.4246, -0.0681]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 7, 2])


### Implementing Multi-Head Attention with Weight Splits

In [44]:
class MultiHeadAttention(nn.Module):
    """
    Listing 3.5 An efficient multi-head attention class.
    """

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):

        super().__init__()
        assert (d_out % num_heads == 0), \
            f"d_out ({d_out}) must be divisible by num_heads ({num_heads})!"

        self.d_out = d_out
        self.num_heads = num_heads

        # Reduces the projection dim to match the desired output dim.
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        # Uses a Linear layer to combine head outputs.
        self.out_proj = nn.Linear(d_out, d_out)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            name="mask",
            tensor=torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):

        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # We implicitly split the matrix by adding a num_heads dimension.
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transposes: 
        #   from  (b, num_tokens, num_heads, head_dim) 
        #   to    (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Computes dot product for each head.
        attn_scores = queries @ keys.transpose(2, 3)

        # Masks truncated to the number of tokens.
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        
        # Uses the mask to fill attention scores.
        attn_scores.masked_fill_(mask_bool, -torch.inf)
    
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Tensor shape: (b, num_tokens, n_heads, head_dim).
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combines heads, where self.d_out = self.num_heads * self.head_dim.
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)

        # Adds an optional linear projection.
        context_vec = self.out_proj(context_vec)

        return context_vec

In [45]:
# The shape of this tensor is (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4).
a = torch.tensor(
    [[[[0.2745, 0.6584, 0.2775, 0.8573],
       [0.8993, 0.0390, 0.9268, 0.7388],
       [0.7179, 0.7058, 0.9156, 0.4340]],
      [[0.0772, 0.3565, 0.1479, 0.5331],
       [0.4066, 0.2318, 0.4545, 0.9737],
       [0.4606, 0.5159, 0.4220, 0.5786]
    ]]]
)

In [46]:
a.shape

torch.Size([1, 2, 3, 4])

In [47]:
b = a @ a.transpose(2, 3)
print(a.shape)
print(a.transpose(2, 3).shape)
print(b.shape)
print(b)

torch.Size([1, 2, 3, 4])
torch.Size([1, 2, 4, 3])
torch.Size([1, 2, 3, 3])
tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


In [48]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [49]:
torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape

d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028],
         [0.2232, 0.5420]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028],
         [0.2232, 0.5420]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 7, 2])


#### Exercise 3.3: Initializing GPT-2 Size Attention Modules

In [50]:
%%time

torch.manual_seed(123)
batch_size = 1; context_length = 1024; d_in = 768; d_out = 768; num_heads = 12; dropout = 0.0

size = (batch_size, context_length, d_in)
params = dict(d_in=d_in, d_out=d_out, context_length=context_length, dropout=dropout, num_heads=num_heads)

batch = torch.empty(size=size)
mha = MultiHeadAttention(**params)
context_vecs = mha(batch)

print("context_vecs.shape:", context_vecs.shape)

context_vecs.shape: torch.Size([1, 1024, 768])
CPU times: user 385 ms, sys: 121 ms, total: 506 ms
Wall time: 326 ms
